In [30]:
## step 1: define the necessary folders 
## (these are mounted volumes from local file system "volgenmodel-docker/data" 
## into the docker container defined in "docker-compose.yml")

import os
os.getcwd()
os.system("ls")
os.chdir("/volgenmodel-nipype")
print(os.getcwd())
os.system("ls")

input_folder = '/opt/conda/bin/data/masked_brains'
output_folder = '/opt/conda/bin/data/output_templates'
process_folder = '/opt/conda/bin/data/converted_brains'
tmp_folder = '/opt/conda/bin/data/tmp'

LICENSE
README.md
__pycache__
extra-scripts
graph.png
new_data_to_atlas_space.py
post_processing.sh
pre_processing.sh
volgenmodel.py
workflow_output_workflowMultiProc50
workflow_temp_workflowMultiProc50
/volgenmodel-nipype
LICENSE
README.md
__pycache__
extra-scripts
graph.png
new_data_to_atlas_space.py
post_processing.sh
pre_processing.sh
volgenmodel.py
workflow_output_workflowMultiProc50
workflow_temp_workflowMultiProc50


In [43]:
##artifical test with test_examples (so they have to convert to nii to do the whole pipeline)
testlist = os.listdir(input_folder)
for t in testlist:
    it = join(input_folder,t)
    cmd = " ".join(["mnc2nii",it, join(input_folder,t.replace("mnc","nii"))])
    os.system(cmd)

found zspace at 0 0
found yspace at 1 1
found xspace at 2 2
0 2 0 1 97 2.500000
1 1 1 1 230 2.500000
2 0 2 1 134 2.500000
found zspace at 0 0
found yspace at 1 1
found xspace at 2 2
0 2 0 1 97 2.500000
1 1 1 1 228 2.500000
2 0 2 1 134 2.500000
found zspace at 0 0
found yspace at 1 1
found xspace at 2 2
0 2 0 1 97 2.500000
1 1 1 1 228 2.500000
2 0 2 1 134 2.500000


In [78]:
## step 2: convert the source files (have to be in a nibabel readable format e.g. nii, nii.gz, if)
## to .mnc as the volgenmodel pipeline can only work with this format 
## (the source nii-files have to be named: gerbilXX.nii)

def nii_to_minc(inputFolder, outputFolder):
  def fileList(folder):
      from os import listdir
      from os.path import isfile, join
      fileListing = [f for f in listdir(folder) 
               if isfile(join(folder, f))]
      fileListing.sort()
      return fileListing    
      
  ## refresh niftiheader; step only neccessariy to prevent bugs in volgen-pipeline
  import nibabel as nib
  import numpy as np
  import os
  main_path = inputFolder
  out_folder = outputFolder
  main_files = fileList(main_path)

  for i, file in enumerate(main_files):
      n1_img = nib.load(join(main_path, file))
      n1_img.header.set_sform(np.diag([1, 1, 1, 1]), code='unknown')
      n1_img.header['dim'][0] = int(3)
      n1_img.header['extents'] = 0
      n1_img.header['sform_code'] = 1
      nib.save(n1_img, join(*[out_folder,'mouse' + str(i) + ".nii"]))
  
  ## convert from nii to minc, because volgenmodel can only use mnc files
    
  mainFiles = fileList(out_folder)
  for file in mainFiles:
      #print(file)
      if file.find('mouse') != -1 and file.find('nii') != -1:
          cmd = " ".join(['nii2mnc',
                '-unsigned',
                '-float',
                '-clobber',
                os.path.join(out_folder, file[:-4]),
                os.path.join(out_folder, file[:-4] + ".mnc")])
          print(cmd)    
          ## followed command could be used to use 32bit float files after 
          ## image normalization (i.e. to cut out parts in source image datasets)
          #os.system('nii2mnc -unsigned -float -clobber ' + out_folder + file[:-4] +
          #' ' + out_folder + file[:-4] + '.mnc')           
          os.system(cmd) 
  
  ## refresh mnc-header; step only neccessariy to prevent bugs in volgen-pipeline
  mainFiles = fileList(out_folder) 
  for file in mainFiles:
      #print(file)
      if file[-3:] == 'mnc':
          
          os.system('mincreshape -clobber -2 +direction' +
                    ' -dimorder zspace,yspace,xspace' +
                    ' -dimsize xspace=-1' +
                    ' -dimsize yspace=-1' +
                    ' -dimsize zspace=-1' + 
                    ' ' + os.path.join(*[out_folder, file]) + ' ' +
                    os.path.join(*[out_folder, file])
                   )
          print(f"{file} -> reworked_mouse_{file}")
    ## delete unused .nii files in processed folder
  

def delete_files(folder, not_ending):
    files = os.listdir(folder)
    for file in files:
        if file[-3:] != not_ending:
            os.system(f"rm {os.path.join(folder,file)}")

nii_to_minc(input_folder, process_folder)
delete_files(process_folder, 'mnc')

nii2mnc -unsigned -float -clobber /opt/conda/bin/data/converted_brains/mouse0 /opt/conda/bin/data/converted_brains/mouse0.mnc
Using s-form transform:
  2.5000,   0.0000,   0.0000, -166.9500, 
  0.0000,   2.5000,   0.0000, -286.2000, 
  0.0000,   0.0000,   2.5000, -119.7000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |  -119.7000 |  2.5000 |  0.0000  0.0000  1.0000
yspace |  -286.2000 |  2.5000 |  0.0000  1.0000  0.0000
xspace |  -166.9500 |  2.5000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.
nii2mnc -unsigned -float -clobber /opt/conda/bin/data/converted_brains/mouse1 /opt/conda/bin/data/converted_brains/mouse1.mnc
Using s-form transform:
  2.5000,   0.0000,   0.0000, -166.9500, 
  0.0000,   2.5000,   0.0000, -284.4000, 
  0.0000,   0.0000,   2.5000, -119.7000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |  -119.7000 |  2.5000 |  0.0000  0.0000  1.0000
yspace |  -284.4000 |  2

<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/opt/conda/bin/data/converted_brains/mouse0.nii'
  image_filename = '/opt/conda/bin/data/converted_brains/mouse0.nii'
  image_offset = '352'
  ndim = '3'
  nx = '134'
  ny = '230'
  nz = '97'
  dx = '2.5'
  dy = '2.5'
  dz = '2.5'
  datatype = '16'
  datatype_name = 'FLOAT32'
  nvox = '2989540'
  nbyper = '4'
  byteorder = 'LSB_FIRST'
  scl_slope = '1'
  scl_inter = '0'
  xyz_units = '2'
  xyz_units_name = 'mm'
  descrip = 'mnc2nii /opt/conda/bin/data/masked_brains/mouse00.mnc /opt/conda/bin/data/maske'
  sform_code = '2'
  sform_code_name = 'Aligned Anat'
  sto_xyz_matrix = '2.5 0 0 -166.95 0 2.5 0 -286.2 0 0 2.5 -119.7 0 0 0 1'
  sto_ijk matrix = '0.4 0 0 66.78 0 0.4 0 114.48 0 0 0.4 47.88 0 0 0 1'
  sform_i_orientation = 'Left-to-Right'
  sform_j_orientation = 'Posterior-to-Anterior'
  sform_k_orientation = 'Inferior-to-Superior'
  num_ext = '0'
/>
<nifti_image
  nifti_type = 'NIFTI-1+'
  header_filename = '/opt/conda/bin/d

Using s-form transform:
  2.5000,   0.0000,   0.0000, -166.9500, 
  0.0000,   2.5000,   0.0000, -284.4000, 
  0.0000,   0.0000,   2.5000, -119.7000, 
  0.0000,   0.0000,   0.0000,   1.0000, 

name   | start      | step    | cosines
zspace |  -119.7000 |  2.5000 |  0.0000  0.0000  1.0000
yspace |  -284.4000 |  2.5000 |  0.0000  1.0000  0.0000
xspace |  -166.9500 |  2.5000 |  1.0000  0.0000  0.0000
Performing voxel format conversion.


Copying chunks:.................................................................................................Done.
Copying chunks:...

mouse0.mnc -> reworked_mouse_mouse0.mnc


..............................................................................................Done.
Copying chunks:.....

mouse1.mnc -> reworked_mouse_mouse1.mnc


............................................................................................

mouse2.mnc -> reworked_mouse_mouse2.mnc


Done.


In [95]:
import os
from itertools import combinations
import os
from datetime import datetime
import os
from os import listdir
from os.path import isfile, join

def process_files(process_folder, output_folder, tmp_folder, result_folder = '/volgenmodel-nipype'):
    # Get all .mnc files in the folder
    files = [f for f in os.listdir(process_folder) if f.endswith('.mnc')]

    # Sort files to ensure order consistency
    files.sort()

    # Iterate over all combination sizes (from 2 to len(files))
    for r in range(2, len(files) + 1):
        for combo in combinations(files, r):
            # Create the output file name
            file_ids = [f.replace("mouse_", "").replace(".mnc", "") for f in combo]
            outputfile = f"temp_{'_'.join(file_ids)}.nii"            
            combo_filelist = [os.path.join(process_folder, f) for f in list(combo)]
            print(combo_filelist)
            # Call the create_template function with the current combination
            create_template(combo_filelist, tmp_folder)
            # convert the template created for this combination
            convert_and_copy_mnc2nii(result_folder, outputfile)
            print(os.system(f"ls {output_folder}"))
    return 0

def create_template(filelist, tmp_folder, result_folder = '/volgenmodel-nipype'):
    tmp_files = os.listdir(tmp_folder)
    if len(tmp_files) > 0:
        print("there's an issue with the tmp folder, it isnt empty!")
        return 1
    else:
        # copy the current set of files that should be used to calculate the template to the tmp folder
        for file in filelist:
            print(f'cp {file} {tmp_folder}')
            os.system(f'cp {file} {tmp_folder}')
        # create the templates
        run_volgenmodel(tmp_folder, ncpus=50, fit_stages='lin,1')
        # clear the tmp-folder for the next template combination
        os.system(f'rm {tmp_folder}/*')
        return 0
                  
def run_volgenmodel(tmp_folder, ncpus=50, fit_stages='lin,1,3'):
    cmd = " ".join(['python3',
                    '/volgenmodel-nipype/volgenmodel.py',
                    '--run=MultiProc',
                    f'--ncpus={ncpus}',
                    f'--fit_stages={fit_stages}',
                    f'--input_dir={tmp_folder}'])               
    os.system(cmd)

def get_last_modified_subfolder(folder_path):
    last_modified_time = None
    last_modified_subfolder = None

    for root, dirs, files in os.walk(folder_path):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            # Get the most recent modification time in the folder (including its files and subfolders)
            for sub_root, _, sub_files in os.walk(dir_path):
                for file_name in sub_files:
                    file_path = os.path.join(sub_root, file_name)
                    file_mod_time = os.path.getmtime(file_path)
                    if last_modified_time is None or file_mod_time > last_modified_time:
                        last_modified_time = file_mod_time
                        last_modified_subfolder = dir_path

    if last_modified_subfolder:
        return last_modified_subfolder, datetime.fromtimestamp(last_modified_time).strftime('%Y-%m-%d %H:%M:%S')
    else:
        return None, None

def convert_and_copy_mnc2nii(folder_to_scan, output_filename):
    subfolder, mod_time = get_last_modified_subfolder(folder_to_scan)
    if subfolder:
        print(f"The last modified subfolder is: {subfolder}")
        print(f"Last modification time: {mod_time}")
    else:
        print("No subfolders found or no modifications detected.")
        return 1  # Return early if no subfolder is found

    # Change subfolder if its name contains "temp" to "output"
    subfolder = subfolder.replace("temp", "output")
    
    # Check if "output" subfolder exists and contains files
    if "output" in subfolder and os.path.exists(subfolder):
        output_files = [f for f in os.listdir(subfolder) if os.path.isfile(os.path.join(subfolder, f))]
        if not output_files:
            print(f"There's no output folder with files: {subfolder}")
            return 1  # Return if the folder exists but is empty
    else:
        print(f"No valid output folder found: {subfolder}")
        return 1  # Return if the folder does not exist

    # Convert every result in the output folder to .nii, even if there are multiple outputs
    cwd = join(subfolder, 'model')
    input_files = [os.path.join(cwd, f) for f in os.listdir(cwd) if os.path.isfile(os.path.join(cwd, f))]
    for i, input_file in enumerate(input_files):
        if i != 0:
            output_file = f'/opt/conda/bin/data/output_templates/{"_".join([str(i), output_filename])}'
        else:
            output_file = f'/opt/conda/bin/data/output_templates/{output_filename}'
        cmd = " ".join(["mnc2nii", input_file, output_file])
        os.system(cmd)

    os.system("ls '/opt/conda/bin/data/output_templates'")

process_files(process_folder, output_folder, tmp_folder)

['/opt/conda/bin/data/converted_brains/mouse00.mnc', '/opt/conda/bin/data/converted_brains/mouse01.mnc']
cp /opt/conda/bin/data/converted_brains/mouse00.mnc /opt/conda/bin/data/tmp
cp /opt/conda/bin/data/converted_brains/mouse01.mnc /opt/conda/bin/data/tmp
250321-12:22:48,150 nipype.workflow INFO:
	 [Node] Setting-up "datasource" in "/tmp/tmpoq86f2yn/datasource".
250321-12:22:48,152 nipype.workflow INFO:
	 [Node] Executing "datasource" <nipype.interfaces.io.SelectFiles>
250321-12:22:48,185 nipype.workflow INFO:
	 [Node] Finished "datasource", elapsed time 0.000485s.

outfiles = ['/opt/conda/bin/data/tmp/mouse00.mnc', '/opt/conda/bin/data/tmp/mouse01.mnc']

+++ Last Linear stage: 0
+++ Fitting
  + [Stage: 00_] End stage: lin
   -- Model Max step: 8.0
---Linear fit---
  + [Stage: 01_] End stage: 1
   -- Model Max step: 4.0
---Non Linear fit---
250321-12:22:48,433 nipype.workflow INFO:
	 Workflow workflow_temp_workflowMultiProc50 settings: ['check', 'execution', 'logging', 'monitoring']
2

0

In [96]:
os.system('ls /volgenmodel-nipype/workflow_output_workflowMultiProc50/model')

mouse00_volcentre_norm_resample_bigaverage_reshape_vol_symm.mnc
mouse01_volcentre_norm_resample_bigaverage_reshape_vol_symm.mnc


0

In [91]:
os.system('rm /volgenmodel-nipype/workflow_output_workflowMultiProc50/ -rf')

0